#### Utility code to convert from CSV to feather for faster load times

# Describe the original dataset

### Instantly recognizable outliers:
* **passenger_count**
    * Passenger counts of 0
    * Infeasible passenger counts (e.g. 200). Taxis can legally hold up to 4-6 passengers (http://www.nyc.gov/html/tlc/html/faq/faq_pass.shtml)
* **fare_amount**
    * Negative fare amounts
    * Extremely high fare amounts (thousands of dollars)
* **coordinates**
    * Invalid coordinates (out of defined range)
    * Coordinates outside of NYC

***
# Data cleaning functions:
#### Check for invalid coordinates (outside of NYC range), determined with https://www.mapdevelopers.com/geocode_bounding_box.php

In [1]:
def valid_coordinates(lat_list, lon_list):
    for i in lat_list:
        if i < 40.477399 or i > 40.917577:
            return False
    for i in lon_list:
        if i < -74.259090 or i > -73.700272:
            return False
    return True

def clean_coordinates(df):
    df.query('~(pickup_latitude < 40.477399 or pickup_latitude > 40.917577) &\
              ~(dropoff_latitude < 40.477399 or dropoff_latitude > 40.917577) &\
              ~(pickup_longitude < -74.259090 or pickup_longitude > -73.700272) &\
              ~(dropoff_longitude < -74.259090 or dropoff_longitude > -73.700272)', inplace=True)

#### Remove invalid passenger counts, fares. Upper bound to both. Also drop missing values and key column.

In [2]:
def clean_pfdk(df):
    df.drop(columns=['key'], inplace=True)
    df.dropna(inplace=True)
    df.query('passenger_count > 0 &\
              passenger_count <= 6 &\
              fare_amount > 0 &\
              fare_amount <= 100 ', inplace=True)

***
# Feature Engineering functions:
***
#### Calculate Euclidean Distance (and drop rows with distance = 0 if training). 

In [3]:
def euclidean_distance(x1, y1, x2, y2):
    return ((x2-x1)**2 + (y2-y1)**2)**0.5

def add_euclidean_distance(df, training=True):
    df['euclidean_distance'] = euclidean_distance(df['pickup_latitude'], 
                                                  df['pickup_longitude'], 
                                                  df['dropoff_latitude'], 
                                                  df['dropoff_longitude'])
    if training:
        df.query('euclidean_distance > 0', inplace=True)
        df.reset_index(drop=True, inplace=True)

#### Calculate Manhattan Distance (and drop rows with distance = 0 if training). 

In [4]:
def manhattan_distance(x1, y1, x2, y2):
    return abs(x2-x1) + abs(y2-y1)

def add_manhattan_distance(df, training=True):
    df['manhattan_distance'] = manhattan_distance(df['pickup_latitude'], 
                                                  df['pickup_longitude'], 
                                                  df['dropoff_latitude'], 
                                                  df['dropoff_longitude'])
    if training:
        df.query('manhattan_distance > 0', inplace=True)
        df.reset_index(drop=True, inplace=True)

#### Because of fixed fares, also calculate distance (euclidean) to/from airports.
* Coordinates determined with a Google Maps query

In [5]:
def add_jfk(df):
    df['go_jfk'] = euclidean_distance(df['dropoff_latitude'], df['dropoff_longitude'], 40.6413, -73.7781)
    df['leave_jfk'] = euclidean_distance(df['pickup_latitude'], df['pickup_longitude'], 40.6413, -73.7781)
def add_lga(df):
    df['go_lga'] = euclidean_distance(df['dropoff_latitude'], df['dropoff_longitude'], 40.7769, -73.8740)
    df['leave_lga'] = euclidean_distance(df['pickup_latitude'], df['pickup_longitude'], 40.7769, -73.8740)
def add_ewr(df):
    df['go_ewr'] = euclidean_distance(df['dropoff_latitude'], df['dropoff_longitude'], 40.6895, -74.1745)
    df['leave_ewr'] = euclidean_distance(df['pickup_latitude'], df['pickup_longitude'], 40.6895, -74.1745)

#### Extract Time Values

In [6]:
def add_times(df):
    df['year']  = df['pickup_datetime'].dt.year
    df['month'] = df['pickup_datetime'].dt.month
    df['day']   = df['pickup_datetime'].dt.day
    df['hour']  = df['pickup_datetime'].dt.hour + (df['pickup_datetime'].dt.minute/60)
    df.drop(columns=['pickup_datetime'], inplace=True)

#### External data: Monthly Gas Prices acquired from https://data.bls.gov/timeseries/APU000074714

In [7]:
def add_gas_prices(df):
    gas_df = pd.read_csv('./all/gas_prices.csv')
    gas_dict = {}
    for row in gas_df.itertuples():
        year = row[1]
        for i in range(2, len(row)):
            gas_dict['{0}-{1}'.format(year, i-1)] = row[i]
    df['year-month']  = df['year'].astype(str) + '-' + df['month'].astype(str)
    df['gas'] = df['year-month'].map(gas_dict)
    df.drop(columns=['year-month'], inplace=True)

#### Preprocess pipeline function

In [20]:
def preprocess(df, training=True):
    if training:
        clean_pfdk(df)
        clean_coordinates(df)
        df.reset_index(drop=True, inplace=True)
    add_euclidean_distance(df, training)
    add_manhattan_distance(df, training)
    add_times(df)
    add_gas_prices(df)
    add_jfk(df)
    add_lga(df)
    add_ewr(df)

#### Utility code to preprocess training data and save as feather

In [9]:
import pandas as pd
import feather

############## PREPROCESS TRAINING DATA $##############
# preprocess(df, training=True)
############## WRITE PREPROCESSED FEATHER #############
# df.to_feather('./all/preprocessed_train.feather')
######### READ PREPROCESSED FEATHER ###################
df = pd.read_feather('./all/preprocessed_train.feather')

***
# Visualization:
#### Scatter plot (Euclidean Distance, Fare Amount)

Observations:
* Very strong correlation (0.87)
* There are three groups of fixed fares over any distance. This makes sense because there are usually fixed fares to JFK, Newark, and LaGuardia airports. 

#### ScatterPlot (Time of Day, Distance Traveled)

Observations:
* Seems to be very low/negligible correlation (-0.03)
* Naturally, less people travel in the early morning

#### ScatterPlot (Time of Day, Fare Amount)

Observations:
* Correlation also very weak (-0.018)
* Three groups of fixed fares also visible here

#### Correlation Heatmap

Observations:
* Euclidean distance has the highest correlation with fare amount
* Manhattan distance is almost perfectly correlated with euclidean
* Distance to/from airports have significant correlation with fare
* Latidude is negatively correlated with fare
* Longitude is positively correlated with fare
* Year has a correlation of 0.12 with fare. Makes sense since costs slowly rise with inflation.
* Gas prices and year have significant correlation (0.4)

#### Fares by Destination

***   
# Train/Test split wrapper function
Can specify number of rows to train on and test subset size.

In [10]:
# SPLIT TRAIN/TEST DATA
from sklearn.model_selection import train_test_split

def get_tt_split(df, test_size=0.2):
    y = df['fare_amount']
    X = df.drop(columns=['fare_amount', 'manhattan_distance'])
    return train_test_split(X, y, test_size=test_size, random_state=3)

***
# Model: Linear Regression

#### Training Results:

Observations:
* Most important feature:
* Scaling has no effect on performance

# Model: Decision Tree Ensembles (XGBoost)

In [11]:
# XGBOOST (DECISION TREE ENSEMBLES)

import xgboost as xgb
from xgboost import DMatrix
from sklearn.metrics import mean_squared_error

def train_xgb(df):
    X_train, X_test, y_train, y_test = get_tt_split(df, test_size=0.005)
    
    print('Training model: {0}...'.format('xgb'))
    model = xgb.train(params={'eta':0.6}, dtrain=DMatrix(X_train, y_train))
    test_predictions = model.predict(xgb.DMatrix(X_test))
    print('Mean Squared Error (Training):', mean_squared_error(y_test, test_predictions))
    
    return model

#### Training Results:

In [12]:
%%time
train_xgb(df)

Training model: xgb...
Mean Squared Error (Training): 10.894632028215339
Wall time: 10min 37s


# Model: Neural Network

In [26]:
# NEURAL NETWORK

import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.train import RMSPropOptimizer
from tensorflow.nn import relu, softmax
from tensorflow.keras.layers import Input, Dense
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

def train_nn(df):
    X_train, X_test, y_train, y_test = get_tt_split(df.head(50000), test_size=0.2)
    X_train = MinMaxScaler().fit_transform(X_train)
    X_test = MinMaxScaler().fit_transform(X_test)
    
    shape = (len(X_train[0]),)
    learning_rate = 0.01
    model = keras.Sequential([Dense(128, input_shape=shape), Dense(128, activation=relu), Dense(1)])
    model.compile(loss='mse', metrics=['mse'], optimizer=RMSPropOptimizer(learning_rate))
    print('Training model: {0}...'.format('nn'))
    model.fit(X_train, y_train, epochs=300, verbose=0)

    test_predictions = [i[0] for i in model.predict(X_test)]
    print('Mean Squared Error (Training):', mean_squared_error(y_test, test_predictions))
    
    return model

#### Training Results:

In [ ]:
%%time
train_nn(df)

Training model: nn...


Observations:
* Outperforms Linear Regression model with only a small fraction of the data
* Takes significantly longer than other models to run
* Scaling improves performance significantly

# Specify model:

In [13]:
selected_model = 'xgb'

# Preprocess/fit model to real test data:

In [21]:
real_df = pd.read_csv('./all/test.csv', parse_dates=['pickup_datetime'])
preprocess(real_df, training=False)

real_X = real_df.drop(columns=['key', 'manhattan_distance'])

if selected_model == 'lr':
    model = train_lr(df)
    predictions = model.predict(real_X)
elif selected_model == 'xgb':
    model = train_xgb(df)
    predictions = model.predict(xgb.DMatrix(real_X))
elif selected_model == 'nn':
    model = train_nn(df)
    real_X = MinMaxScaler().fit_transform(real_X)
    predictions = [i[0] for i in model.predict(real_X)]

data = list(zip(real_df['key'], predictions))

submission = pd.DataFrame(data, columns=['key', 'fare_amount'])
submission.set_index('key', inplace=True)
submission.to_csv('submission.csv')
print('----------------------------')
print('Predictions written to submissions.csv')

----------------------------
Predictions written to submissions.csv


## Best Kaggle Scores by model (RMSE):
* Linear Regression: 5.64822
* Neural Network: 4.36043
* Decision Tree Ensembles (XGBoost): 3.36031   <---